In [73]:
import numpy as np
import matplotlib.pyplot as plt
from math import sin, cos, pi, exp


In [101]:
# Определение функции
def func(x, y):
    return x**2 + y**2 #sin(x) * cos(y)
def func_a(x, y):
    return 3 * (1 - x)**2 * exp(-x**2 - (y + 1)**2) - 10 * (x / 5 - x**3 - y**5) * exp(-x**2 - y**2) - 1/3 * exp(-(x + 1)**2 - y**2)
def func_b(x, y):
    return -0.0001 * (abs(sin(x) * sin(y) * exp(abs(100 - ((x**2 + y**2)**0.5) / pi))) + 1)**0.1

# Параметры генетического алгоритма
population_size = 1000
gene_length = 10  # Длина хромосомы для каждой переменной
crossover_rate = 0.7
mutation_rate = 0.01
generations = 1000

# Диапазон значений переменных
x_min, x_max = -10, 10
y_min, y_max = -10, 10


In [136]:
def initialize_population(size, dimensions=2):
    return np.random.rand(size, dimensions)

def decode(chromosome):
    x = x_min + (x_max - x_min) * chromosome[0]
    y = y_min + (y_max - y_min) * chromosome[1]
    return x, y

def fitness_maximization(chromosome):
    x, y = decode(chromosome)
    return max(0, given_func(x, y))  # Гарантируем, что значение приспособленности неотрицательно

def fitness_minimization(chromosome):
    x, y = decode(chromosome)
    value = given_func(x, y)
    return 1 / (1 + value) if value > 0 else 1 + abs(value)

def selection(population):
    fitness_values = np.array([fit_func(ind) for ind in population])
    probabilities = fitness_values
    probabilities = (probabilities - (probabilities.min() if (probabilities.min() != np.nan) and (any(probabilities != probabilities.min())) else 0))
    probabilities = probabilities / (probabilities.sum() if probabilities.sum() != 0 else 1)
    chosen_indices = np.random.choice(range(population_size), size=population_size, replace=True, p=probabilities)
    return population[chosen_indices]

def crossover(parent1, parent2):
    if np.random.rand() < crossover_rate:
        point = np.random.randint(0, len(parent1))
        child1 = np.concatenate((parent1[:point], parent2[point:]))
        child2 = np.concatenate((parent2[:point], parent1[point:]))
        return child1, child2
    else:
        return parent1, parent2

def mutate(chromosome):
    for i in range(len(chromosome)):
        if np.random.rand() < mutation_rate:
            chromosome[i] = np.random.rand()
    return chromosome


In [137]:
population = initialize_population(population_size)
fit_func = fitness_maximization
given_func = func_a
for generation in range(generations):
    new_population = []
    population = selection(population)
    for i in range(0, population_size, 2):
        parent1, parent2 = population[i], population[i+1]
        child1, child2 = crossover(parent1, parent2)
        child1 = mutate(child1)
        child2 = mutate(child2)
        new_population.extend([child1, child2])
    population = np.array(new_population)

# Определение лучшего решения
best_individual = max(population, key=fit_func)
best_x, best_y = decode(best_individual)
best_fitness = given_func(best_x, best_y)

print(f"Лучшие значения переменных: x = {best_x}, y = {best_y}")
print(f"Максимальное значение функции: {best_fitness}")


Лучшие значения переменных: x = -0.005874824721116667, y = 1.5878983913017297
Максимальное значение функции: 8.10543784067372


In [134]:
population = initialize_population(population_size)
fit_func = fitness_minimization
given_func = func_b
for generation in range(generations):
    new_population = []
    population = selection(population)
    for i in range(0, population_size, 2):
        parent1, parent2 = population[i], population[i+1]
        child1, child2 = crossover(parent1, parent2)
        child1 = mutate(child1)
        child2 = mutate(child2)
        new_population.extend([child1, child2])
    population = np.array(new_population)

# Определение лучшего решения
best_individual = max(population, key=fit_func)
best_x, best_y = decode(best_individual)
best_fitness = given_func(best_x, best_y)

print(f"Лучшие значения переменных: x = {best_x}, y = {best_y}")
print(f"Минимальное значение функции: {best_fitness}")


Лучшие значения переменных: x = -1.3186841684016866, y = -1.2362363170234243
Минимальное значение функции: -2.061033083174423
